In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
MODEL_NAME = os.getenv("MODEL_NAME") or ""
BASE_URL = os.getenv("BASE_URL") or ""
API_KEY = os.getenv("API_KEY") or "" 
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""

# LLM

In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

c:\Users\pkmpp\OneDrive\Documents\GitHub\AI-Projects\Python\RAG\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

website (blog site)

In [5]:
# import bs4
# from langchain_community.document_loaders import WebBaseLoader

# # only keep post title, headers, and content form the full HTML
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
# loader = WebBaseLoader(
#     # web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     web_paths=("https://docs.langchain.com/oss/python/integrations/chat/ollama",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )
# docs = loader.load()

In [6]:
# print(docs[-1].metadata)

In [7]:
# print(docs[-1].page_content)

text splitter

In [8]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

loader = PyMuPDF4LLMLoader(
    file_path="C:/Users/pkmpp/Downloads/1.pdf"
)
docs = await loader.aload()

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [9]:
# print(docs[-1].page_content)

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

# print(f"Split blog post into {len(all_splits)} sub-documents.")




In [11]:
print(all_splits[-1].page_content)

A retail store wants to introduce an automated discount calculation system to improve
customer billing efficiency.
The discount offered to a customer depends on the total purchase amount.


As a junior data analyst, you are asked to write a simple R program to determine the

applicable discount percentage for a customer based on the purchase amount entered.


Write an R program that performs the following tasks:


1. Prompt the user to enter the total purchase amount.


2. Convert the entered value into a numeric data type.


3. Use if–else if–else conditional statements to determine the discount according to

|the rules below:|Col2|
|---|---|
|Purchase Amount (LKR)|Discount|
|Amount ≥ 5000|20%|
|Amount ≥ 3000|15%|
|Amount ≥ 1000|10%|
|Amount < 1000|5%|



4. Display the applicable discount percentage clearly.


Classification: Public-SLIIT


In [12]:
document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids[:3])

['87c5b10a-edf5-4280-8b50-2ffe1da43f77']


In [13]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [14]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, SecretStr
from langchain.agents import create_agent
# from components import prompt_with_context

llm = ChatOpenAI(model=MODEL_NAME, base_url=BASE_URL, api_key=SecretStr(API_KEY))
# tools = [retrieve_context]
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(
    model=llm, tools=[], system_prompt=prompt, middleware=[prompt_with_context]
)

for step in agent.stream(
    {"messages":"get the full document that the content that start with this `A retail store wants to introduce an automated discount calculation system to improve customer billing efficiency.` "},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

get the full document that the content that start with this `A retail store wants to introduce an automated discount calculation system to improve customer billing efficiency.` 
================================== Ai Message ==================================

Sure! Here is the complete document based on the content you provided:

---

**A retail store wants to introduce an automated discount calculation system to improve customer billing efficiency.**

The discount offered to a customer depends on the total purchase amount.

As a junior data analyst, you are asked to write a simple R program to determine the applicable discount percentage for a customer based on the purchase amount entered.

### Requirements:
1. Prompt the user to enter the total purchase amount.
2. Convert the entered value into a numeric data type.
3. Use `if–else if–else` conditional statements to determine the discount according to th

In [15]:
step

{'messages': [HumanMessage(content='get the full document that the content that start with this `A retail store wants to introduce an automated discount calculation system to improve customer billing efficiency.` ', additional_kwargs={}, response_metadata={}, id='d8986939-ba36-47cc-994b-baa7949d7a0f'),
  AIMessage(content='Sure! Here is the complete document based on the content you provided:\n\n---\n\n**A retail store wants to introduce an automated discount calculation system to improve customer billing efficiency.**\n\nThe discount offered to a customer depends on the total purchase amount.\n\nAs a junior data analyst, you are asked to write a simple R program to determine the applicable discount percentage for a customer based on the purchase amount entered.\n\n### Requirements:\n1. Prompt the user to enter the total purchase amount.\n2. Convert the entered value into a numeric data type.\n3. Use `if–else if–else` conditional statements to determine the discount according to the fo

In [3]:
from pathlib import Path
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)


files = [f.name for f in UPLOAD_DIR.iterdir() if f.is_file()]
print(files)


['IT3031-Practical1.pdf']


In [8]:
# 1. Get the list of names
files = [f.name for f in UPLOAD_DIR.iterdir() if f.is_file()]

# 2. Pass them into the loader (joining the path)
for file_name in files:
    full_path = str(UPLOAD_DIR / file_name)
    loader = PyMuPDF4LLMLoader(file_path=full_path)
    docs = await loader.aload()

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [9]:
docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2021-02-12T15:25:01+05:30', 'source': 'uploads\\IT3031-Practical1.pdf', 'file_path': 'uploads\\IT3031-Practical1.pdf', 'total_pages': 2, 'format': 'PDF 1.7', 'title': 'IT3031 - Practical1', 'author': 'Prasanna', 'subject': '', 'keywords': '', 'moddate': '2021-02-12T15:25:01+05:30', 'trapped': '', 'modDate': "D:20210212152501+05'30'", 'creationDate': "D:20210212152501+05'30'", 'page': 0}, page_content='**IT3031 - Database Systems and Data Driven Applications**\n\nBSc (Hons) in IT Data Science\n\nFaculty of Computing\nSri Lanka Institute of Information Technology\n\nPractical 1 – Relational Model\n\n1. Use the following relational database schema for recording the information shown below.\n\nPrimary keys are underlined.\n\n\nEmployee (EmpNo: varchar(20), fname:char(20), lname:char(20),\naddress:varchar(40),salary:integer, DeptNo:varchar(20))\n\n\nDepartmen

In [6]:
files

['IT3031-Practical1.pdf']

In [8]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
load_dotenv()

from pathlib import Path
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)


pdf_paths = [str(UPLOAD_DIR / f.name) for f in UPLOAD_DIR.iterdir() if f.suffix == ".pdf"]
print(f"Loading files: {pdf_paths}")


MODEL_NAME = os.getenv("MODEL_NAME") or ""
BASE_URL = os.getenv("BASE_URL") or ""
API_KEY = os.getenv("API_KEY") or "" 
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""


embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)


async def load_all_pdfs(paths):
    all_docs = []
    for path in paths:
        loader = PyMuPDF4LLMLoader(file_path=path)
        # aload() returns a list of documents for one PDF
        pdf_docs = await loader.aload()
        all_docs.extend(pdf_docs)
    return all_docs

docs = await load_all_pdfs(pdf_paths)
print(docs)

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

document_ids = vector_store.add_documents(documents=all_splits)


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

Loading files: ['uploads\\IT3030-PAF_Lec01-Frameworks.pdf']
[Document(metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2025-02-17T02:28:12+05:30', 'source': 'uploads\\IT3030-PAF_Lec01-Frameworks.pdf', 'file_path': 'uploads\\IT3030-PAF_Lec01-Frameworks.pdf', 'total_pages': 16, 'format': 'PDF 1.7', 'title': 'Introduction to PAF', 'author': 'Vishan Jayasinghearachchi', 'subject': '', 'keywords': '', 'moddate': '2025-06-01T23:50:31+05:30', 'trapped': '', 'modDate': "D:20250601235031+05'30'", 'creationDate': "D:20250217022812+05'30'", 'page': 0}, page_content='# SOFTWARE FRAMEWORKS – AN OVERVIEW\n\nPROGRAMMING APPLICATIONS AND FRAMEWORKS (IT3030)\n\n\n'), Document(metadata={'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'creationdate': '2025-02-17T02:28:12+05:30', 'source': 'uploads\\IT3030-PAF_Lec01-Frameworks.pdf', 'file_path': 'uploads\\

In [24]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import asyncio
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
load_dotenv()

from pathlib import Path
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)



MODEL_NAME = os.getenv("MODEL_NAME") or ""
BASE_URL = os.getenv("BASE_URL") or ""
API_KEY = os.getenv("API_KEY") or "" 
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""


embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)


async def load_all_pdfs(paths):
    all_docs = []
    for path in paths:
        loader = PyMuPDF4LLMLoader(file_path=path)
        # aload() returns a list of documents for one PDF
        pdf_docs = await loader.aload()
        all_docs.extend(pdf_docs)
    return all_docs


embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

async def main():
    pdf_paths = [str(UPLOAD_DIR / f.name) for f in UPLOAD_DIR.iterdir() if f.suffix == ".pdf"]
    print(f"Loading files: {pdf_paths}")

    docs = await load_all_pdfs(pdf_paths)
    print(f"Loaded {len(docs)} pages")

    all_splits = text_splitter.split_documents(docs)
    print(f"Created {len(all_splits)} chunks")

    document_ids = vector_store.add_documents(documents=all_splits)
    print(f"Added {len(document_ids)} documents to vector store")


if __name__ == "__main__":
    asyncio.run(main())


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings
load_dotenv()

from pathlib import Path
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)




MODEL_NAME = os.getenv("MODEL_NAME") or ""
BASE_URL = os.getenv("BASE_URL") or ""
API_KEY = os.getenv("API_KEY") or "" 
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""


embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)


async def load_all_pdfs(paths):
    all_docs = []
    for path in paths:
        loader = PyMuPDF4LLMLoader(file_path=path)
        # aload() returns a list of documents for one PDF
        pdf_docs = await loader.aload()
        all_docs.extend(pdf_docs)
    return all_docs


pdf_paths = [str(UPLOAD_DIR / f.name) for f in UPLOAD_DIR.iterdir() if f.suffix == ".pdf"]
print(f"Loading files: {pdf_paths}")
docs = await load_all_pdfs(pdf_paths)

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids)


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

Loading files: ['uploads\\IT3030-PAF_Lec01-Frameworks.pdf']


c:\Users\pkmpp\OneDrive\Documents\GitHub\AI-Projects\Python\RAG\backend\.venv\Lib\site-packages\pymupdf\__init__.py:24036: RuntimeWarning: coroutine 'initialize_vector_store' was never awaited
  def handle_args():


['83d03838-50b3-4eb5-aa55-1fb072b65880', '26221e0e-8370-4f50-b761-0646d5722a87', '94e3c815-5aa7-4cab-b907-54ca78276313', '66cd36cb-d930-4138-8b59-73dd991728f0', 'aa00523c-4f79-4884-ad81-e742079ae960', '69bb3dd9-e08d-435a-ab41-c87d31e69ca0', '5c9ffa1d-73b2-40ae-87d8-f324bfcf3599', '5158fd05-d24c-4164-8b53-e186bd8a3502', 'be516997-623c-4022-815d-0c6100e1b8cb', '6792f73d-14a9-4ada-a47e-58b49e27ddc3', '1c12f4f6-41f7-4949-b10a-abb64954d06a', '00fbab2f-4abb-41f6-97e8-85a2ccc0bae3', '7e8b3329-e694-446a-bb7e-f4061b748948', '74a878a6-c966-475c-ae4a-cbeae428a42e', '426de11b-b8d9-4dc9-9203-87f527164bd7']


In [3]:
import os
import asyncio
from pathlib import Path
from dotenv import load_dotenv

# LangChain & Vector Store Imports
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.agents.middleware import dynamic_prompt, ModelRequest

# Load environment variables
load_dotenv()

# Configuration
UPLOAD_DIR = Path("uploads")
UPLOAD_DIR.mkdir(exist_ok=True)

MODEL_NAME = os.getenv("MODEL_NAME") or ""
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""

# Initialize Embeddings
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

# Global variable to hold our vector store after loading
vector_store = None

async def load_all_pdfs(paths):
    """Iterates through paths and loads PDF content asynchronously."""
    all_docs = []
    for path in paths:
        print(f"Reading: {path}")
        loader = PyMuPDF4LLMLoader(file_path=path)
        # aload() returns a list of document objects for that specific PDF
        pdf_docs = await loader.aload()
        all_docs.extend(pdf_docs)
    return all_docs

async def initialize_vector_store():
    """Initializes the knowledge base once."""
    global vector_store
    
    # 1. Gather all PDF paths
    pdf_paths = [str(f) for f in UPLOAD_DIR.iterdir() if f.suffix.lower() == ".pdf"]
    
    if not pdf_paths:
        print("⚠️ No PDF files found in 'uploads' folder.")
        return None

    # 2. Load the documents
    docs = await load_all_pdfs(pdf_paths)
    
    # 3. Split text into manageable chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True
    )
    all_splits = text_splitter.split_documents(docs)
    
    # 4. Create the FAISS vector store
    print(f"Creating index for {len(all_splits)} text chunks...")
    vector_store = FAISS.from_documents(documents=all_splits, embedding=embeddings)
    print("✅ Vector store is ready.")
    return vector_store

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages from the pre-built vector store."""
    initialize_vector_store()
    global vector_store
    
    # Safety check if vector_store hasn't finished loading
    if vector_store is None:
        return "You are a helpful assistant. (Knowledge base not available)"

    # Get the user's latest message
    last_query = request.state["messages"][-1].text
    
    # Search for relevant snippets
    retrieved_docs = vector_store.similarity_search(last_query, k=3)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # Return the system prompt with injected context
    return (
        "You are a helpful assistant. Use the following context to answer the user's question. "
        "If the answer isn't in the context, tell them you don't know based on the documents.\n\n"
        f"CONTEXT:\n{docs_content}"
    )

# # --- Execution Entry Point ---
# if __name__ == "__main__":
#     # This runs the initialization logic and starts the event loop
#     asyncio.run(initialize_vector_store())
    
#     # After this, your framework (like LangChain Agents) can take over
#     print("System is idle and waiting for queries...")

In [4]:
document_ids

NameError: name 'document_ids' is not defined

In [3]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, SecretStr
from langchain.agents import create_agent

llm = ChatOpenAI(model=MODEL_NAME, base_url=BASE_URL, api_key=SecretStr(API_KEY))
# tools = [retrieve_context]
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(
    model=llm, tools=[], system_prompt=prompt, middleware=[prompt_with_context]
)

for step in agent.stream(
    {"messages":"give me a summery of Module Offering form  "},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give me a summery of Module Offering form  
================================== Ai Message ==================================

The **Module Offering Form** is a document used by educational institutions to record and manage the **courses or modules** that are offered in a specific **academic semester**. It is essential for planning, scheduling, and tracking academic offerings for students and staff.

### Key Components of the Module Offering Form:
1. **Module Code**: Unique identifier for the module.
2. **Module Title**: Name of the course or module.
3. **Credits**: Total number of credits awarded for the module.
4. **Instructor**: Name of the faculty member teaching the module.
5. **Semester**: Indicates the academic semester (e.g., 2026-2027, 2026-2027).
6. **Class Time**: Details of the class schedule (e.g., 08:00-10:00).
7. **Location**: Venue where the module is held (e.g., Lecture Hall 1, Seminar Roo